## Vertical photos combining

In [1]:
import random
from usefull_functions import total_score, create_vertical_slides, load_photos_from_json

In [2]:
v_photos = load_photos_from_json("../data/vertical_photos.json")
# v_photos = v_photos[:10000]

### Random Stitching

In [ ]:
def random_pair_vertical_photos(vertical_photos):
    """
    Returns list of random (photo1, photo2) pairs.
    """

    photos = vertical_photos.copy()

    if len(photos) % 2 == 1:
        photos = photos[:-1]

    random.shuffle(photos)

    pairs = []
    for i in range(0, len(photos), 2):
        pairs.append((photos[i], photos[i + 1]))

    return pairs


In [4]:
v_slides = create_vertical_slides(random_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [27585, 51045], 'tags': {'tb6', 'tf', 't61', 'tq2', 'th5', 'ts2', 'tz4', 'tc5', 't47', 't85', 't87', 'tk1', 'tz6', 'tl5', 'tb7'}}, {'photos': [81354, 68757], 'tags': {'t31', 'tv2', 't91', 'tp2', 'tz2', 'tq1', 'tx', 'tm5', 'tf1', 'tq6', 'tp6', 't53', 'tw5', 't74', 't82', 'tc5', 'tm2', 'tw3', 't52', 'tz6', 't25', 'tl5', 't86', 'th4', 't05'}}, {'photos': [72513, 4765], 'tags': {'tp2', 'tx1', 'tr5', 'tq6', 't53', 'tl5', 't93', 'th4', 't82', 'tc6', 'tw2', 'tl3', 't34', 'tc5', 't85', 'tb5', 't52', 't6', 't91', 'tz6', 't37'}}, {'photos': [64991, 9925], 'tags': {'t82', 'tz2', 'th6', 't85', 't02', 'tm5', 'tw3', 'tz', 't01', 'tl5', 'th4', 'tw5'}}, {'photos': [74455, 38072], 'tags': {'tq2', 'th5', 'tn6', 'tl2', 't87', 't02', 'tr5', 'tb5', 't6', 'tz6', 't01', 'tw6', 'tp6', 'th4'}}, {'photos': [52975, 58490], 'tags': {'tp2', 'th5', 'tr5', 't01', 'tp6', 'tw5', 'tf3', 't82', 't34', 'tq3', 't32', 'tl2', 't47', 't6', 'tz6', 'tl5', 't05'}}, {'photos': [88677, 16581], 'tags': {'tw5', 't82', '

In [5]:
score = total_score(v_slides)
print(score)

146361


### Similar photos

In [37]:
def similar_pair_vertical_photos(vertical_photos, k=50):
    """
    A fast greedy implementation for large datasets.
    k: number of candidates based on the number of tags for each photo
    """
    photos = sorted(vertical_photos, key=lambda x: len(x["tags"]))
    if len(photos) % 2 == 1:
        photos = photos[:-1]

    used = set()
    pairs = []
    n = len(photos)

    for idx, photo in enumerate(photos):
        if photo["id"] in used:
            continue

        start = max(0, idx - k)
        end = min(n, idx + k)
        best_score = -1
        best_pair = None

        for j in range(start, end):
            candidate = photos[j]
            if candidate["id"] in used or candidate["id"] == photo["id"]:
                continue
            score = len(photo["tags"] & candidate["tags"])
            if score > best_score:
                best_score = score
                best_pair = candidate

        if best_pair is not None:
            pairs.append((photo, best_pair))
            used.add(photo["id"])
            used.add(best_pair["id"])

    return pairs


In [38]:
v_slides = create_vertical_slides(similar_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [805, 4053], 'tags': {'t34', 'tz2', 't52'}}, {'photos': [991, 3078], 'tags': {'tw3', 't47', 'td4'}}, {'photos': [1060, 4567], 'tags': {'t53', 't85', 'ts2'}}, {'photos': [1740, 7716], 'tags': {'tr5', 'tq2', 't01'}}, {'photos': [2259, 14328], 'tags': {'tx', 'th5', 'tz4'}}, {'photos': [2800, 23469], 'tags': {'tm3', 't05', 'tj1'}}, {'photos': [4069, 10305], 'tags': {'t31', 'tm2', 'tk5'}}, {'photos': [6215, 29216], 'tags': {'tr5', 'tq1', 't47'}}, {'photos': [7411, 13751], 'tags': {'t87', 'th6', 'th4'}}, {'photos': [8528, 11256], 'tags': {'t57', 't87', 'tq6'}}]


In [39]:
score = total_score(v_slides)
print(score)

120637


### Different photos

In [3]:
def different_pair_vertical_photos(vertical_photos, k=300):
    """
    Combines photos with minimal tag intersection.
    k: Number of candidates for finding the minimal intersection
    """
    photos = sorted(vertical_photos, key=lambda x: len(x["tags"]))
    if len(photos) % 2 == 1:
        photos = photos[:-1]

    used = set()
    pairs = []
    n = len(photos)

    for idx, photo in enumerate(photos):
        if photo["id"] in used:
            continue

        start = max(0, idx - k)
        end = min(n, idx + k)
        best_score = float('inf')
        best_pair = None

        for j in range(start, end):
            candidate = photos[j]
            if candidate["id"] in used or candidate["id"] == photo["id"]:
                continue
            score = len(photo["tags"] & candidate["tags"])
            if score < best_score:
                best_score = score
                best_pair = candidate

        if best_pair is not None:
            pairs.append((photo, best_pair))
            used.add(photo["id"])
            used.add(best_pair["id"])

    return pairs

In [4]:
v_slides = create_vertical_slides(different_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [805, 991], 'tags': {'tz2', 't52', 'td4', 't47'}}, {'photos': [1060, 1740], 'tags': {'ts2', 't01', 'tr5', 't53'}}, {'photos': [2259, 2800], 'tags': {'tz4', 'tj1', 'th5', 't05'}}, {'photos': [3078, 4053], 'tags': {'t34', 'tw3', 'td4', 'tz2'}}, {'photos': [4069, 4567], 'tags': {'tm2', 't31', 't53', 't85'}}, {'photos': [6215, 7411], 'tags': {'t47', 'tq1', 'th6', 'th4'}}, {'photos': [7716, 8528], 'tags': {'t87', 't57', 'tq2', 'tr5'}}, {'photos': [9251, 10305], 'tags': {'tn6', 'tw3', 'tm2', 'tk5'}}, {'photos': [10565, 11256], 'tags': {'tc4', 'tz2', 'tq6', 't87'}}, {'photos': [11702, 13751], 'tags': {'t87', 'tz2', 'th6', 'tl3'}}]


In [5]:
score = total_score(v_slides)
print(score)

209063
